In [1]:
# code snippet from example 90
F5= GF(5)
F5t.<t> = F5[]
P_MOD_2 = F5t(t^2+2)
P_MOD_2.is_irreducible()

F5_2.<t> = GF(5^2, name='t', modulus=P_MOD_2)
E1F5_2 = EllipticCurve(F5_2,[1,1])
E1F5_2.order()

27

## Exercise 76. 

Consider the Short Weierstrass elliptic curve $E(F_5^2)$ from example 90, compute the expression $(4t + 3, 2t + 1) \oplus (3t + 3, 2)$ using pen and paper and double-check the computation usingsage. Then solve the equation $x \oplus (3t+3, 3) =(3,4)$ for some $x \in E(F_5^2)$. After that compute the scalar multiplication [5](2t + 1, 4t + 4) using the double-and-add algorithm from exercise 38.

**solution:**

### a)
$(4t + 3, 2t + 1) \oplus (3t + 3, 2) $ 


In [2]:
sol1 = E1F5_2(4 * t + 3, 2 * t + 1) + E1F5_2(3 * t + 3, 2)

print(f"(4t + 3, 2t + 1) \oplus (3t + 3, 2) = {sol1}")



(4t + 3, 2t + 1) \oplus (3t + 3, 2) = (0 : 1 : 1)


<>:3: SyntaxWarning: invalid escape sequence '\o'
<>:3: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_264733/1676077228.py:3: SyntaxWarning: invalid escape sequence '\o'
  print(f"(4t + 3, 2t + 1) \oplus (3t + 3, 2) = {sol1}")


### b)
$x \oplus (3t+3, 3) =(3,4) \rightarrow x = (3, 4) \oplus (3t+3, 3)^{-1}$

In [3]:


### c)
point1 = E1F5_2(2*t + 1, 4*t + 4)
double = point1 + point1
cuatro = double + double

assert cuatro + point1 == 5 * point1


## Exercise 77. 

Consider the Tiny-jubjub curve from example 71. Show that the polynomial $t^4 + 2 \in F_{13}[t]$ is irreducible. Then write a sage program to implement the finite field extension $F_{13}^4$ , implement the curve extension TJJ_13($F_{13}^4$) and compute the number of curve points.


In [4]:
F13= GF(13)
F13t.<t> = F13[]
P_MOD_4 = F13t(t^4 + 2)
P_MOD_4.is_irreducible()

F13_4.<t> = GF(13^4, name='t', modulus=P_MOD_4)
TJJ_F5_4 = EllipticCurve(F13_4, [8,8])
TJJ_F5_4.order()

28800

## Exercise 78. 

Consider the alt_bn128 curve and its associated base field Fp from example 73. As we know from example 89 this curve has an embedding degree of 12. Use Sage to find an irreducible polynomial P ∈ Fp[t] and write a sage program to implement the finite field extension Fp12 and to implement the curve extension alt_bn128(Fp12) and compute the number of curve points.


In [5]:
alt_bn128_p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
F_altbn = GF(alt_bn128_p)
F_altbn_t.<t> = PolynomialRing(F_altbn)

degree = 12  # for example, change this to the degree you want
# Find an irreducible polynomial of the specified degree over GF(p)
irreducible_poly = F_altbn_t.irreducible_element(degree)
assert irreducible_poly.is_irreducible()
print(f"irreducible polynomial of degree {degree}: {irreducible_poly}")

F_altbn_12.<t> = GF(alt_bn128_p^degree, name='t', modulus=irreducible_poly)
altbn_128_12 = EllipticCurve(F_altbn_12, [0,3])
altbn_128_12.order()

#### Actual alt_bn_128 extension tower
# Fp 
G1 = EllipticCurve(F_altbn, [0,3])
r = G1.order()

# Fp2
_.<x> = PolynomialRing(F_altbn)
Fp2.<u> = F_altbn.extension(x^2 + 1)

non_residue = u + 9

# Fp6
_.<t> = Fp2[]
Fp6.<v> = Fp2.extension(t^3 - non_residue)

# # Commented as it takes too long
# Fp12
# _.<y> = Fp6[]
# Fp12.<w> = Fp6.extension(y^2 - v)

irreducible polynomial of degree 12: t^12 + 7*t^11 + 15*t^10 + t^9 + 21888242871839275222246405745257275088696311157297823662689037894645226208558*t^8 + 24*t^7 + 152*t^6 + 208*t^5 + 184*t^4 + 144*t^3 + 78*t^2 + 51*t + 71


## Exercise 79. 

Consider the full 5-torsion group T JJ_13[5] from example 92. Write down the set of all elements from this group and identify the subset of all elements from TJJ_13(F13)[5] as well as T JJ_13(F132 )[5]. Then compute the 5-torsion group T JJ_13(F138 )[5] .


In [6]:
def r_torsion_group_TJJ(p, k_r, r):
    F_p = GF(p)
    F_p_t.<t> = F_p[]
    F_p_kr_poly = F_p_t.irreducible_element(k_r)
    assert F_p_kr_poly.is_irreducible()

    F_tjj_kr.<t> = GF(p^k_r, name='t', modulus=F_p_kr_poly)
    TJJ_kr = EllipticCurve(F_tjj_kr, [8,8])
    assert TJJ_kr.order() % r == 0

    INF = TJJ_kr(0)
    L_TJJ_r = INF.division_points(r)
    
    result = Set(L_TJJ_r)
    print(f"F({p}_{k_r})[{r}]: |{len(result)}| {result}")
    print('*' * 20)
    return result

torsion_deg1 = r_torsion_group_TJJ(13, 1, 5)
torsion_deg2 = r_torsion_group_TJJ(13, 2, 5)
torsion_deg4 = r_torsion_group_TJJ(13, 4, 5)
torsion_deg8 = r_torsion_group_TJJ(13, 8, 5)


F(13_1)[5]: |5| {(7 : 11 : 1), (0 : 1 : 0), (8 : 8 : 1), (7 : 2 : 1), (8 : 5 : 1)}
********************
F(13_2)[5]: |5| {(7 : 11 : 1), (0 : 1 : 0), (8 : 8 : 1), (7 : 2 : 1), (8 : 5 : 1)}
********************
F(13_4)[5]: |25| {(t^3 + 5*t^2 + 3*t + 7 : t^3 + 9*t^2 + 6*t : 1), (0 : 1 : 0), (7*t^3 + 7*t^2 + 2*t + 1 : t^3 + 2*t^2 + 4*t + 10 : 1), (3*t^3 + t^2 + t + 6 : 7*t^3 + 9*t^2 + t + 8 : 1), (6*t^3 + 6*t^2 + 11*t + 10 : 10*t^3 + 4*t^2 + 4*t + 3 : 1), (8 : 5 : 1), (9*t^3 + 2*t^2 + 3*t + 3 : 11*t^3 + 2*t^2 + 5*t + 1 : 1), (5*t^2 + 6*t + 11 : 4*t^3 + 2*t^2 + 10 : 1), (9*t^3 + 9*t^2 + 10*t + 9 : 9*t^3 + 11*t^2 + 4*t + 2 : 1), (3*t^3 + t^2 + t + 6 : 6*t^3 + 4*t^2 + 12*t + 5 : 1), (8 : 8 : 1), (3*t^3 + 4*t^2 + t + 1 : 4*t^3 + t + 10 : 1), (9*t^3 + 9*t^2 + 10*t + 9 : 4*t^3 + 2*t^2 + 9*t + 11 : 1), (10*t^3 + 9*t^2 + 12*t : 2*t^3 + 9*t^2 + 5*t + 12 : 1), (7*t^3 + 7*t^2 + 2*t + 1 : 12*t^3 + 11*t^2 + 9*t + 3 : 1), (t^3 + 5*t^2 + 3*t + 7 : 12*t^3 + 4*t^2 + 7*t : 1), (7 : 11 : 1), (5*t^2 + 6*t + 11

## Exercise 80. 

Consider the curve secp256k1 from example 72 and its full r-torsion group as introduced in example 93. Write down a single element from the curves full torsion group that is not the point at infinity.


In [7]:
F_secp = GF(115792089237316195423570985008687907853269984665640564039457584007908834671663)
secp256k1 = EllipticCurve(F_secp, [0, 7])
r = secp256k1.cardinality().factor()[-1][0]
INF = secp256k1(0)

# from chatgpt
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
P = secp256k1(Gx, Gy)

assert r * P == INF

## Exercise 81.

Consider the curve alt_bn128 from example 73 and its curve extension from exercise 78. Write a Sage program that computes a generator from the curves full torsion group.

In [8]:
altbn_128 = EllipticCurve(F_altbn, [0,3])
r = altbn_128.order()
assert r.is_prime()

# generator point take from chatgpt
G1 = altbn_128(1, 2)
INF = altbn_128(0)
assert G1 * r == INF

## Exercise 82.

Consider the small prime factor 2 of the Tiny-jubjub curve. Compute the full 2-torsion group of $TJJ_{13}$ and then compute the groups $\mathbb{G}_1[2]$ and $\mathbb{G}_2[2]$.

In [9]:
TJJ_13 = EllipticCurve(F13, [8,8])
TJJ_tor2 = r_torsion_group_TJJ(13, 1, 2)

def frobenious_end(P):
    if P == TJJ_13(0):
        return P
    else:
        x,y = P.xy()
        return TJJ_13(x^13, y^13)

G1_2 = [p for p in TJJ_tor2 if frobenious_end(p) == p] 
print(f"G1: {G1_2}")
G2_2 = [p for p in TJJ_tor2 if frobenious_end(p) == 13 * p]
print(f"G2: {G2_2}")

F(13_1)[2]: |2| {(4 : 0 : 1), (0 : 1 : 0)}
********************
G1: [(4 : 0 : 1), (0 : 1 : 0)]
G2: [(4 : 0 : 1), (0 : 1 : 0)]


## Exercise 83.

Consider the curve alt_bn128 from example 73 and its curve extension from exercise 78. Write a Sage program that computes a generator for each of the torsion group G1[p] and G2[p].

**Solution:**

From exercise 81, we got the generator of G1, now we need to find out the generator in G2

In [10]:
### Info take from this link: https://hackmd.io/@vivi432/bn128-in-c

F2 = GF(alt_bn128_p^2, "i" ,modulus=x^2 + 1)
TwistB = 3*F2("9+i")^(-1)
twisted_bn = EllipticCurve(F2,[0, TwistB])

G2x = F2("11559732032986387107991004021392285783925812861821192530917403151452391805634*i + 10857046999023057135944570762232829481370756359578518086990519993285655852781")
G2y = F2("4082367875863433681332203403145435568316851327593401208105741076214120093531*i + 8495653923123431417604973247489272438418190587263600148770280649306958101930")
G2 = twisted_bn(G2x, G2y)
assert G2 * twisted_bn.order() == twisted_bn(0)

## Exercise 84.

Consider the curve alt_bn128 from example 73 and the generators g1 and g2 of G1[p] and G2[p] from exercise 83. Write a Sage program that computes the Weil pairing e(g1,g2).